In [15]:
import re
import os
import pandas as pd

In [16]:
def list_files():
    files = os.listdir('reuters/reuters21578')
    reuters_file_pattern = re.compile('reut2-[\d]+.sgm')
    reuters_files = []
    for file in files:
        if reuters_file_pattern.match(file) == None:
            continue
        reuters_files.append(file)
    return reuters_files

In [17]:
def get_data():
    reuters_files = list_files()
    reuters_data = []
    for file in reuters_files:
        fobj = open('reuters/reuters21578/' + file, mode='r', encoding='utf8', errors='ignore')
        data = fobj.read()
        fobj.close()
        data = data.replace('\n', ' ')
        reuters_data.append(data)
    return reuters_data

In [18]:
categories = ['corn', 'cotton', 'rice', 'soybean', 'wheat']

In [19]:
def load_data():
    reuters_pattern = re.compile('(<REUTERS.*?>.*?</REUTERS>)')
    pattern = re.compile('(<TOPICS>.*?</TOPICS>).*?(<TEXT.*?>.*?</TEXT>)')
    topic_pattern = re.compile('<D>(.*?)</D>')
    text_pattern = re.compile('<TITLE>(.*?)</TITLE>.*?<BODY>(.*?)</BODY>')

    reuters_data = {'doc_id': [], 'text':[], 'topics':[]}
    reuters_data_list = get_data()

    for data in reuters_data_list:
        reuters = reuters_pattern.findall(data)
        id = 0
        for reuter in reuters:
            content = pattern.findall(reuter)
            topics = topic_pattern.findall(content[0][0])
            if len(topics) == 0:
                continue
    #         off_topics = [topic for topic in topics if topic not in categories]
    #         if len(off_topics) != 0:
            in_topics = [topic for topic in topics if topic in categories]
            if len(in_topics) == 0:
                continue
            text_list = text_pattern.findall(content[0][1])
            if len(text_list) == 0:
                continue
            text = ' '.join(text_list[0])
            id += 1
            reuters_data['doc_id'].append(id)
            reuters_data['topics'].append(in_topics)
            reuters_data['text'].append(text)
    return reuters_data

In [20]:
reuters_data = load_data()
reuters_df = pd.DataFrame(reuters_data)
reuters_df.to_csv('reuters.csv', index=False)

In [21]:
# creates csv in binary topic format
data_topics = reuters_data['topics']
topics_dict = {'corn': [], 'cotton': [], 'rice': [], 'soybean': [], 'wheat': []}
for topics in data_topics:
    for key in topics_dict:
        if key in topics:
            topics_dict[key].append(1)
        else:
            topics_dict[key].append(0) 
topics_df = pd.DataFrame(topics_dict)

In [27]:
reuters_df.drop('topics', axis = 1, inplace = True)

In [35]:
reuters_new_df = pd.concat([reuters_df, topics_df], axis = 1)
reuters_new_df.head()
reuters_new_df.to_csv('reuters_new.csv', index=False)